In [22]:
import pandas as pd
import numpy as np
from sklearn import cross_validation
from sklearn.linear_model import LinearRegression

## Load training set

In [23]:
train = pd.read_csv("../data/train.csv", dtype={'StateHoliday': str}, parse_dates=["Date"])
train = train.loc[train.Sales > 0]
train['month'] = train['Date'].dt.month
train['day'] = train['Date'].dt.day
columns = ['DayOfWeek'] #['Store', 'DayOfWeek', 'Promo']

## Cross validate median

In [14]:
(cv_train, cv_test) = cross_validation.train_test_split(train)
median = cv_train['Sales'].median()
cv_medians = cv_train.groupby( columns )['Sales'].median()
cv_medians = cv_medians.reset_index()
cv_medians.rename(columns={'Sales': 'Sales2'}, inplace=True)
cv_test = pd.merge( cv_test, cv_medians, on = columns, how = 'left' )
cv_medians['Sales2'].fillna(median)

spe = ((cv_test['Sales']-cv_test['Sales2'])/cv_test['Sales'])**2.0
mspe = spe.sum()/len(spe)
rmspe = np.sqrt(mspe)
print("RMSPE=%f" % rmspe)

RMSPE=0.505686


## Sparse matrix

In [31]:
(cv_train, cv_test) = cross_validation.train_test_split(train)
dow = cv_train['DayOfWeek']
X = pd.DataFrame({'1' : dow == 1, '2': dow == 2, '3': dow == 3, 
                  '4': dow == 4, '5': dow == 5, '6': dow == 6, '7': dow == 7})
dow2 = cv_test['DayOfWeek']
X2 = pd.DataFrame({'1' : dow2 == 1, '2': dow2 == 2, '3': dow2 == 3, 
                  '4': dow2 == 4, '5': dow2 == 5, '6': dow2 == 6, '7': dow2 == 7})
classifier = LinearRegression()
classifier.fit(X, cv_train['Sales'])
y = classifier.predict(X2)


spe = ((cv_test['Sales']-y)/cv_test['Sales'])**2.0
mspe = spe.sum()/len(spe)
rmspe = np.sqrt(mspe)
print("RMSPE=%f" % rmspe)

RMSPE=0.586477


Nic to nie dało. Mediana ciągle jest lepszym modelem. Takie podejście chyba bliżej jest średniej